In [1]:
!pip install --upgrade tensorflow
!pip install tf-quant-finance
!pip install tqdm

In [2]:
#@title Imports { display-mode: "form" }

import matplotlib.pyplot as plt
import numpy as np
import time

import tensorflow as tf

 # tff for Tensorflow Finance
import tf_quant_finance as tff

from IPython.core.pylabtools import figsize
figsize(21, 14) # better graph size for Colab  

from tqdm.notebook import tqdm, trange

In [3]:
#@title Set up parameters

dtype = np.float64 #@param
num_samples = 10#200000 #@param

num_timesteps = 2#100 #@param

expiries = [1.0] # This can be a rank 1 Tensor
dt = 1. / num_timesteps
rate = tf.constant(0.03, dtype=dtype)
sigma = tf.constant(0.1, dtype=dtype)
spot = tf.constant(700, dtype=dtype)

def set_up_pricer(expiries, watch_params=False):
    """Set up European option pricing function under Black-Scholes model.
    
    Args:
        expiries: List of expiries at which to to sample the trajectories.
        watch_params: A Python bool. When `True`, gradients of the price function wrt the inputs
          are computed more efficiently. 
    Returns:
     A callable that accepts a rank 1 tensor of strikes, and scalar values for 
     the spots and  volatility values. The callable outputs prices of
     the European call options on the grid `expiries x strikes`.
    """
    def price_eu_options(strikes, spot, sigma):
        # Define drift and volatility functions. 
        def drift_fn(t, x):
          del t, x
          return rate - 0.5 * sigma**2
        def vol_fn(t, x):
          del t, x
          return tf.reshape(sigma, [1, 1])
        # Use GenericItoProcess class to set up the Ito process
        process = tff.models.GenericItoProcess(
            dim=1,
            drift_fn=drift_fn,
            volatility_fn=vol_fn,
            dtype=dtype)
        log_spot = tf.math.log(tf.reduce_mean(spot))
        if watch_params:
            watch_params_list = [sigma]
        else:
            watch_params_list = None
        paths = process.sample_paths(
            expiries, num_samples=num_samples,
            initial_state=log_spot, 
            watch_params=watch_params_list,
            # Select a random number generator
            random_type=tff.math.random.RandomType.PSEUDO_ANTITHETIC,
            time_step=dt)
        prices = (tf.exp(-tf.expand_dims(rate * expiries, axis=-1))
                  * tf.reduce_mean(tf.nn.relu(tf.math.exp(paths) - strikes), 0))
        return prices
    return price_eu_options

price_eu_options = tf.function(set_up_pricer(expiries),
                    input_signature=[tf.TensorSpec([None], dtype=tf.float64),
                     tf.TensorSpec([], dtype=tf.float64),
                     tf.TensorSpec([], dtype=tf.float64)])
price_eu_options2 = tf.function(set_up_pricer(expiries))

In [4]:
print(price_eu_options.pretty_printed_concrete_signatures())

price_eu_options(strikes, spot, sigma)
  Args:
    strikes: float64 Tensor, shape=(None,)
    spot: float64 Tensor, shape=()
    sigma: float64 Tensor, shape=()
  Returns:
    float64 Tensor, shape=(1, None)


In [5]:

print(price_eu_options2.pretty_printed_concrete_signatures())

In [6]:
strikes = tf.constant(np.random.rand(5) * (0.5*spot) + spot, dtype )
print(price_eu_options(strikes, spot, sigma))
print(price_eu_options2(strikes, spot, sigma))

tf.Tensor([[ 0.          0.          6.8511844  20.64492577  0.        ]], shape=(1, 5), dtype=float64)
tf.Tensor([[ 0.          0.          7.67145896 25.48659649  0.        ]], shape=(1, 5), dtype=float64)


In [7]:
print(price_eu_options.pretty_printed_concrete_signatures())

price_eu_options(strikes, spot, sigma)
  Args:
    strikes: float64 Tensor, shape=(None,)
    spot: float64 Tensor, shape=()
    sigma: float64 Tensor, shape=()
  Returns:
    float64 Tensor, shape=(1, None)


In [8]:
print(price_eu_options2.pretty_printed_concrete_signatures())

price_eu_options(strikes, spot, sigma)
  Args:
    strikes: float64 Tensor, shape=(5,)
    spot: float64 Tensor, shape=()
    sigma: float64 Tensor, shape=()
  Returns:
    float64 Tensor, shape=(1, 5)


In [9]:
for i in trange(1, 100000):
    strikes = tf.constant(np.random.rand(i) * (0.5*spot) + spot, dtype )
    prices = price_eu_options(strikes, spot, sigma)
    if i%100:
        print(prices[:10])

  0%|          | 0/99999 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[[ 0.          6.15336132 19.70186585 ...  0.          0.
   0.        ]], shape=(1, 97903), dtype=float64)
tf.Tensor(
[[ 1.91063023  0.         21.93312556 ...  0.          1.67029206
   7.05477156]], shape=(1, 97904), dtype=float64)
tf.Tensor([[0. 0. 0. ... 0. 0. 0.]], shape=(1, 97905), dtype=float64)
tf.Tensor(
[[ 0.          0.         45.94604072 ... 12.91471204  0.
   6.14134304]], shape=(1, 97906), dtype=float64)
tf.Tensor(
[[ 0.          0.          0.         ...  0.         16.85974947
   7.7787429 ]], shape=(1, 97907), dtype=float64)
tf.Tensor(
[[ 0.         28.38537391  0.         ...  0.         47.95133121
  22.4614575 ]], shape=(1, 97908), dtype=float64)
tf.Tensor(
[[36.88198445  0.          0.         ...  0.          0.
   0.        ]], shape=(1, 97909), dtype=float64)
tf.Tensor(
[[ 0.          0.          0.         ... 28.93190439  0.11379248
  29.26708083]], shape=(1, 97910), dtype=float64)
tf.Tensor([[0. 0. 0. ... 

In [10]:
print(price_eu_options.pretty_printed_concrete_signatures())

price_eu_options(strikes, spot, sigma)
  Args:
    strikes: float64 Tensor, shape=(None,)
    spot: float64 Tensor, shape=()
    sigma: float64 Tensor, shape=()
  Returns:
    float64 Tensor, shape=(1, None)
